# Evaluating the Models

## This Juypter Notebook was created to easily load the models and set/change the directories to test the models, all in one place.

In [1]:
import os
from database import Database
from models.altered_xception import AlteredXception
from models.faster_r_cnn import FasterRCNN
from models.rmac_model import RMACModel
from utils import grab_images_and_paths, grab_all_image_paths 

/Users/lchris/.pyenv/versions/3.9.1/lib/python3.9/site-packages/gluoncv/__init__.py:40: UserWarning: Both `mxnet==1.9.1` and `torch==1.8.1` are installed. You might encounter increased GPU memory footprint if both framework are used at the same time.
  warnings.warn(f'Both `mxnet=={mx.__version__}` and `torch=={torch.__version__}` are installed. '


## Loading The Models and Data

In [2]:
# NOTE: if the RMAC model needs to get descriptions of many different square regions, it will take a long time to load/initialize (roughly a minute).
ax_model = AlteredXception()
rmac_model = RMACModel(ax_model.model.get_layer("conv2d_3").output_shape[1:], 3)
rcnn_model = FasterRCNN()
models = [ax_model, rmac_model, rcnn_model]

2022-11-23 17:06:10.069180: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/models/rmac_model.py:120: RuntimeWarning: invalid value encountered in double_scalars
  b = (W - wl) / (l + Wd - 1)
/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/models/rmac_model.py:125: RuntimeWarning: invalid value encountered in double_scalars
  b = (H-wl)/(l+Hd-1)


In [3]:
# test_dir is the directory where our "database" of test images/frames and query images should exist. 
test_dir = "/Users/lchris/Desktop/Coding/schoolprojects/comp490/COMPS/eval/test_images/test_games"
frame_paths, frames = grab_images_and_paths(os.path.join(test_dir, "frames"))
query_image_paths = grab_all_image_paths(os.path.join(test_dir, "query_images"))

In [4]:
# Get the YouTube video "ground truth" from the IDs we picked, stored in a .txt file
ground_truth_ids = []
ground_truth_file_path = os.path.join(test_dir, "ground_truth.txt")
with open(ground_truth_file_path, "r") as ground_truth:
    for line in ground_truth.readlines():
        youtube_ground_truth_id = line.split("/")[-1].split(".")[0]
        ground_truth_ids.append(youtube_ground_truth_id)
ground_truth_ids

['-tTaYmNB-IM',
 'q_hZHqNnwAU',
 'r3FaHBe1rPw',
 'S7Nqx_pyEl0',
 'cmh9FnLbE5s',
 'huGVGe3Afng',
 'kQnQrYmhXW8',
 '_lAtb14VGm8',
 '0J57bBrDDLc',
 '5X6SctBSd9A']

In [5]:
# Initialize models to query from the evaluation database
database = Database(os.path.join(test_dir, "predictions"))
for model in models:
    model.database = database

In [6]:
# Gathering predictions for each of the frames in the dataset
for model in models:
    print(model)
    if (isinstance(model, AlteredXception)) and len(database.prediction_image_paths) < len(frame_paths):
        model_predictions = model.predict_images(frames)
        database.store_predictions(model_predictions, frame_paths)
    if (isinstance(model, RMACModel)) and len(database.rmac_prediction_image_paths) < len(frame_paths):
        model_predictions = model.predict_images(frames)
        database.store_rmac_predictions(model_predictions, frame_paths)
    if (isinstance(model, FasterRCNN)) and len(database.object_predictions) < len(frame_paths):
        model_predictions = model.predict_image_paths(frame_paths)
        database.store_object_data(model_predictions, frame_paths)

: 

: 

In [ ]:
def post_process_predictions(predictions):
    video_id_preds = []
    for prediction in predictions:
        image_path = prediction[0].split("/")[-1] # We only want the last part of the path
        video_id = image_path[0:11]
        video_id_preds.append(video_id)
    return video_id_preds

In [ ]:
results = {}
for query_image_path in query_image_paths:
    results[query_image_path] = {}
    for model in models:
        model_name = type(model).__name__
        curr_predictions = post_process_predictions(model.query_image(query_image_path)[0:len(ground_truth_ids)])
        results[query_image_path][model_name] = curr_predictions
results

In [ ]:
matches = {}
for query_image_path in query_image_paths:
    curr_results = results[query_image_path]
    matches[query_image_path] = {}
    for model in results[query_image_path]:
        num_matches = 0
        for id in ground_truth_ids:
            if id in results[query_image_path][model]:
                num_matches += 1
        matches[query_image_path][model] = num_matches